# Times for data access via sliderule with save to geoparquet and reopen vs. stream to memory
\
On CryoCloud:
Looks like these are highly variable and either one can be a second faster than the other in timeit

In [23]:
import numpy as np
from sliderule import icesat2
import sliderule
import pyproj
import geopandas as gpd
sliderule.set_verbose(False)
import tempfile
import os

In [20]:
%%timeit
xy0 = [0., -3.0e6]
crs=3413
W=4.e4
pad=np.array([-W/2, W/2])
bbox = [xy0[0] + pad[[0, 1, 1, 0, 0]], xy0[1] + pad[[0, 0, 1, 1, 0]]]
# generate a clipping polygon in a lat/lon coordinate system
bbox_ll = pyproj.Transformer.from_crs(pyproj.CRS(crs), 
                                      pyproj.CRS(4326))\
                    .transform(*bbox)
poly = [{"lat":lat, "lon":lon} for lat, lon in zip(*bbox_ll)]

with tempfile.TemporaryDirectory() as tmpdir:
    save_file = f'{tmpdir}/ATL06_E{int(xy0[0]/1.e3)}_N{int(xy0[1]/1.e3)}.parquet'
    icesat2.atl06sp(
       {
        "poly": poly,
        "t0":"2023-09-13T00:00:00Z",
        "t1":"2023-12-13T00:00:00Z", 
       
        "output": {
          "format": "parquet",
          "as_geo": True,
          "path": save_file,
          "with_checksum": False,
        }})
    df=gpd.read_parquet(save_file).to_crs(3413)

5.74 s ± 2.74 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
%%timeit
xy0 = [0., -3.0e6]
crs=3413
W=4.e4
pad=np.array([-W/2, W/2])
bbox = [xy0[0] + pad[[0, 1, 1, 0, 0]], xy0[1] + pad[[0, 0, 1, 1, 0]]]
# generate a clipping polygon in a lat/lon coordinate system
bbox_ll = pyproj.Transformer.from_crs(pyproj.CRS(crs), 
                                      pyproj.CRS(4326))\
                    .transform(*bbox)
poly = [{"lat":lat, "lon":lon} for lat, lon in zip(*bbox_ll)]

df = icesat2.atl06sp(
   {
    "poly": poly,
    "t0":"2023-09-13T00:00:00Z",
    "t1":"2023-12-13T00:00:00Z", 
   })

5.59 s ± 1.61 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
